In [54]:
import yfinance as yf
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import ipywidgets as widgets
from IPython.display import display, clear_output

def RSI(arr, period=14):
    series = pd.Series(arr)
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi.values

In [55]:
class SmaCross(Strategy):
    n1 = 50
    n2 = 200

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.position.close()

class RsiStrategy(Strategy):
    period = 14
    overbought = 70
    oversold = 30

    def init(self):
        close = self.data.Close
        self.rsi = self.I(RSI, close, self.period)

    def next(self):
        if self.rsi[-1] < self.oversold and not self.position.is_long:
            self.buy()
        elif self.rsi[-1] > self.overbought and self.position.is_long:
            self.position.close()

In [56]:
def run_backtest(ticker='AAPL', start='2020-01-01', end='2023-01-01', strategy_name='SMA'):
    print(f"Downloading data for {ticker} from {start} to {end} ...")
    data = yf.download(ticker, start=start, end=end, auto_adjust=True)
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)

    if strategy_name.upper() == 'SMA':
        strategy = SmaCross
    elif strategy_name.upper() == 'RSI':
        strategy = RsiStrategy
    else:
        print(f"Unknown strategy {strategy_name}, using SMA by default.")
        strategy = SmaCross

    print(f"Running backtest for {strategy_name} strategy...")
    bt = Backtest(data, strategy, cash=10_000, commission=0.001)
    stats = bt.run()
    print(stats)
    bt.plot(resample=False, superimpose=False)

In [57]:
ticker_widget = widgets.Text(value='AAPL', description='Ticker:', style={'description_width': 'initial'})
start_widget = widgets.Text(value='2020-01-01', description='Start Date:', style={'description_width': 'initial'})
end_widget = widgets.Text(value='2023-01-01', description='End Date:', style={'description_width': 'initial'})
strategy_widget = widgets.Dropdown(
    options=['SMA Crossover', 'RSI'],
    value='SMA Crossover',
    description='Strategy:',
    style={'description_width': 'initial'}
)
run_button = widgets.Button(description="Run Backtest", button_style='success')

def on_run_button_clicked(b):
    run_backtest(
        ticker_widget.value,
        start_widget.value,
        end_widget.value,
        strategy_widget.value
    )

run_button.on_click(on_run_button_clicked)

# Display widgets
display(ticker_widget, start_widget, end_widget, strategy_widget, run_button)

Text(value='AAPL', description='Ticker:', style=DescriptionStyle(description_width='initial'))

Text(value='2020-01-01', description='Start Date:', style=DescriptionStyle(description_width='initial'))

Text(value='2023-01-01', description='End Date:', style=DescriptionStyle(description_width='initial'))

Dropdown(description='Strategy:', options=('SMA Crossover', 'RSI'), style=DescriptionStyle(description_width='…

Button(button_style='success', description='Run Backtest', style=ButtonStyle())